In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'password'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Real-time Product Popularity Tracking

## Description of the use case
This use case involves tracking the popularity of products in real-time based on user interactions such as views, likes, and purchases. By monitoring these metrics, businesses can identify trending products and adjust their strategies accordingly.

## Redis Data Structures Used
- **HyperLogLog**: To estimate the unique count of interactions.
- **Hashes**: To store detailed interaction data for each product.
- **Sorted Sets**: To rank products based on their popularity scores.

In [2]:
r.flushdb()

import random
import uuid
mock_data = [f"product_{uuid.uuid4()}" for x in range(0, random.randint(0,10))]
print(f"{len(mock_data)} number of products are selected")
for x in mock_data:
    print(f"Product ID : {x}")


# Use HyperLogLog to estimate the unique count of interactions
print(f"Generating Random number of views, likes and purchases for products")
for product_id in mock_data:
    views = random.randint(0,10000)
    likes = random.randint(0,200)
    purchases = random.randint(0,20)
    expected_score = views + (likes * 2) + (purchases * 5)
    print(f"Expected Score for {product_id} --> {expected_score}")
    
    p = r.pipeline()
    for _ in range(views):
        p.pfadd(f"{product_id}:views", f"user{_}") 
    p.execute()
    for _ in range(likes):
        p.pfadd(f"{product_id}:likes", f"user{_}")
    p.execute() 
    for _ in range(purchases):
        p.pfadd(f"{product_id}:purchases", f"user{_}")
    p.execute()

# Calculate popularity score and use Sorted Sets to rank products
for product_id in mock_data:
    views = r.pfcount(f"{product_id}:views")
    likes = r.pfcount(f"{product_id}:likes")
    purchases = r.pfcount(f"{product_id}:purchases")
    popularity_score = views + (likes * 2) + (purchases * 5)  # Example scoring algorithm
    r.zadd('product_popularity', {product_id: popularity_score})


# Print the ranked products by popularity
ranked_products = r.zrange('product_popularity', 0, -1, withscores=True, desc=True)
print('Ranked Products by Popularity:')
for product, score in ranked_products:
    print(f"Actual Score for {product} --> {score}")

2 number of products are selected
Product ID : product_826d2b35-ce08-48e0-81e5-de3a0dffcf00
Product ID : product_efce608a-92cd-4fd7-9a20-068dacf3c2eb
Generating Random number of views, likes and purchases for products
Expected Score for product_826d2b35-ce08-48e0-81e5-de3a0dffcf00 --> 9716
Expected Score for product_efce608a-92cd-4fd7-9a20-068dacf3c2eb --> 8690
Ranked Products by Popularity:
Actual Score for product_826d2b35-ce08-48e0-81e5-de3a0dffcf00 --> 9763.0
Actual Score for product_efce608a-92cd-4fd7-9a20-068dacf3c2eb --> 8720.0
